In [34]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [35]:
base = pd.read_csv('iris.csv')
base.head(3)

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


In [36]:
previsores = base.iloc[:, 0:4].values
previsores[0:3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [37]:
classe = base.iloc[:, 4].values
classe[0:3]

array(['Iris-setosa', 'Iris-setosa', 'Iris-setosa'], dtype=object)

In [38]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
classe = labelencoder.fit_transform(classe)
classe[0:3]

array([0, 0, 0])

In [39]:
classe_dummy = np_utils.to_categorical(classe)
classe_dummy[0:3]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [40]:
import gc
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import Callback

In [41]:
class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        k.clear_session()

In [42]:
def criar_rede():
    # cleanup
    k.clear_session()
    
    classificador = Sequential()
    
    classificador.add(Dense(units = 4, activation = 'relu', input_dim = 4))
    
    classificador.add(Dense(units = 4, activation = 'relu'))
    
    classificador.add(Dense(units = 3, activation = 'softmax'))
    
    classificador.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                          metrics = ['categorical_accuracy'])
    
    return classificador

In [43]:
classificador = KerasClassifier(build_fn = criar_rede,
                                epochs = 500,
                                batch_size = 10)
                                #callbacks= ClearMemory() )

In [44]:
from sklearn.utils import parallel_backend
import time

In [45]:
start = time.time()
with parallel_backend('threading'):
    resultados = cross_val_score(estimator = classificador,
                         X = previsores, y = classe,
                         cv = 5, scoring = 'f1_weighted',
                         n_jobs = 1)

Epoch 1/500
12/12 [==============================] - 0s 916us/step - loss: 2.9835 - categorical_accuracy: 0.1667
Epoch 2/500
12/12 [==============================] - 0s 999us/step - loss: 2.5266 - categorical_accuracy: 0.1667
Epoch 3/500
12/12 [==============================] - 0s 833us/step - loss: 2.1283 - categorical_accuracy: 0.1667
Epoch 4/500
12/12 [==============================] - 0s 999us/step - loss: 1.7473 - categorical_accuracy: 0.1583
Epoch 5/500
12/12 [==============================] - 0s 913us/step - loss: 1.4662 - categorical_accuracy: 0.0000e+00
Epoch 6/500
12/12 [==============================] - 0s 999us/step - loss: 1.2480 - categorical_accuracy: 0.1250
Epoch 7/500
12/12 [==============================] - 0s 999us/step - loss: 1.1219 - categorical_accuracy: 0.4417
Epoch 8/500
12/12 [==============================] - 0s 999us/step - loss: 1.0546 - categorical_accuracy: 0.5500
Epoch 9/500
12/12 [==============================] - 0s 915us/step - loss: 1.0289 - categori

In [48]:
end = time.time()
total = (end - start)/60
print('{:.2f}min'.format(total))

3.22min


In [49]:
resultados

array([1.        , 0.        , 0.88888889, 1.        , 0.90909091])

In [50]:
media = resultados.mean()
desvio = resultados.std()

In [51]:
print('accuracy mean: {}'.format(media))
print('accuracy Desvio: {}'.format(desvio))

accuracy mean: 0.7595959595959596
accuracy Desvio: 0.38252833322757296
